In [1]:
__author__ = 'Elliot I. Simon'
__email__ = 'ellsim@dtu.dk'
__version__ = 'April 08, 2021'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import glob
import statsmodels.api as sm

In [3]:
%matplotlib inline
#%matplotlib notebook

In [4]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
plt.rcParams['figure.figsize'] = [10, 8]

# Load/prepare WindScanner data

In [ ]:
# Get list of filenames of already merged wind_data
filenames=[]
path = r"R:\Globe\Lidar_data\Onshore_testing\LOS_calibration\Levante\*.txt"
for file in glob.glob(path):
    filenames.append(file)

In [ ]:
# Single file
file = "R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\Sterenn\\batch1_merged.txt"
filenames = [file]

In [ ]:
filenames

In [ ]:
%%time
# Read all files
df_file = (pd.read_csv(file, sep=';', header=None) for file in filenames)
df = pd.concat(df_file, ignore_index=True, axis=0)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Rename azim and elevation columns
df = df.rename(columns={5: 'dt_stop', 6: 'azim', 7: 'elev'})

In [ ]:
# Function to convert labview to datetime stamp
# Time is stamped at beginning of 10min period in UTC format!!!
def convtime(labviewtime):
    unixtime = labviewtime - 2082844800
    timestamp = datetime.datetime.utcfromtimestamp(unixtime) 
    # - pd.Timedelta(minutes=10)
    # + pd.Timedelta(hours=1)
    return timestamp

In [5]:
# Function for going between range gate distance and column number
def get_col_num(range_gate):
    colnum = int((((range_gate - 200)/15)*4)+8)
    return colnum

In [ ]:
# Apply time conversion to dt_stop value (end of acquisition)
time_index = df['dt_stop'].apply(lambda x: convtime(x))
df = df.set_index(time_index)

In [ ]:
df.head()

In [ ]:
df.memory_usage()

In [ ]:
# Interpolate to 1-s stamp
df = df.resample('1S').mean().interpolate().dropna()

In [ ]:
# Save DataFame to disk
df.to_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\Levante\\processed\\levante_los_calib_batch2_1s.hdf', 
          'df', complib='blosc')

In [ ]:
# Plot time index to see gaps in measurement data
plt.plot(df.index)

In [ ]:
# TimeDelta between first and last measurement
df.last_valid_index() - df.first_valid_index()

In [ ]:
range_gates

In [ ]:
# Plot and make sure azimuth + elevation don't change (for LOS)
df['azim'].plot()
df['elev'].plot()

# Load saved data

In [ ]:
df_s = pd.read_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\Sterenn\\processed\\sterenn_los_calib_batch2_1s.hdf', 
          'df')

In [ ]:
df_l = pd.read_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\Levante\\processed\\levante_los_calib_batch2_1s.hdf', 
          'df')

In [ ]:
# Get list of range gates from column values
range_gates_s = df_s.iloc[0:1,8::4].values[0].tolist()
range_gates_l = df_l.iloc[0:1,8::4].values[0].tolist()

In [ ]:
range_gates_s

# Check Measurements

In [ ]:
# Set column for respective scanner
sterenn_rg_col = get_col_num(2000)
levante_rg_col = get_col_num(2000)

In [ ]:
# First look at RadSpeeds
f,axarr = plt.subplots(2,1, sharex=True)
plt.sca(axarr[0])
df_l.iloc[:,levante_rg_col+1].plot()
plt.title('Unfiltered Radial Speed')
plt.ylabel('m/s');
plt.sca(axarr[1])
df_s.iloc[:,sterenn_rg_col+1].plot()

## Filter CNR

In [ ]:
# Loops through list of range gates. Applies CNR mask to set radspeed to NaN if CNR < ul or CNR > ll. 
# Also applies RS mask in similar fashion
cnr_ll = -27
cnr_ul = 0
rs_ll = -17
rs_ul = 17

for dist in range_gates_s:
    rg = get_col_num(dist)
    mask = df_s[rg+2] < cnr_ll
    df_s.loc[mask, rg+1] = np.nan

    mask = df_s[rg+2] > cnr_ul
    df_s.loc[mask, rg+1] = np.nan
    
    mask = df_s[rg+1] < rs_ll
    df_s.loc[mask, rg+1] = np.nan
    
    mask = df_s[rg+1] > rs_ul
    df_s.loc[mask, rg+1] = np.nan

In [ ]:
# Loops through list of range gates. Applies CNR mask to set radspeed to NaN if CNR < ul or CNR > ll. 
# Also applies RS mask in similar fashion
cnr_ll = -27
cnr_ul = 0
rs_ll = -17
rs_ul = 17

for dist in range_gates_l:
    rg = get_col_num(dist)
    mask = df_l[rg+2] < cnr_ll
    df_l.loc[mask, rg+1] = np.nan

    mask = df_l[rg+2] > cnr_ul
    df_l.loc[mask, rg+1] = np.nan
    
    mask = df_l[rg+1] < rs_ll
    df_l.loc[mask, rg+1] = np.nan
    
    mask = df_l[rg+1] > rs_ul
    df_l.loc[mask, rg+1] = np.nan

In [ ]:
f,axarr = plt.subplots(2,1, sharex=True)
plt.sca(axarr[0])
df_l.iloc[:,levante_rg_col+1].plot()
plt.title('Filtered Radial Speed')
plt.ylabel('m/s');
plt.sca(axarr[1])
df_s.iloc[:,sterenn_rg_col+1].plot()

In [ ]:
df_join = pd.concat([df_l.iloc[:,levante_rg_col+1], df_s.iloc[:,sterenn_rg_col+1]], axis=1, join='inner')
df_join.columns = ['levante', 'sterenn']
df_join.dropna(inplace=True)

In [ ]:
df_join['diff'] = df_join['levante'] - df_join['sterenn']

In [ ]:
df_join.shape

In [ ]:
df_join.head()

In [ ]:
df_join.to_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\joined_2000m_batch2.hdf', 
          'df', complib='blosc')

# Export data for Mike

In [34]:
df_join = pd.read_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\joined_2000m_batch1.hdf', 
          'df')
df_join2 = pd.read_hdf('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\joined_2000m_batch2.hdf', 
          'df')

In [35]:
df = pd.concat([df_join, df_join2], axis=0)

In [36]:
df.head()

,levante,sterenn,diff
dt_stop,,,
2021-03-12 15:05:33,-2.729000,-2.570000,-0.159000
2021-03-12 15:05:34,-2.947000,-2.939000,-0.008000
2021-03-12 15:05:35,-2.948000,-3.522000,0.574000
2021-03-12 15:05:36,-3.323000,-3.780000,0.457000
2021-03-12 15:05:37,-4.231000,-4.089000,-0.142000


In [37]:
# Filter diff > 3m/s
df = df[np.abs(df['diff']) < 3].dropna()

In [38]:
# Filter only negative RS
df = df[df['levante'] < 0].dropna()

In [39]:
df.shape

(818758, 3)

In [40]:
df_30s = df.resample('30S').mean().dropna()
df_30s.to_csv('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\Mike\\globe_loscal_2000m_30s.csv',
             sep=';', decimal='.', header=True)

In [41]:
df_1min = df.resample('1Min').mean().dropna()
df_1min.to_csv('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\Mike\\globe_loscal_2000m_1min.csv',
             sep=';', decimal='.', header=True)

In [42]:
df_10min = df.resample('10Min').mean().dropna()
df_10min.to_csv('R:\\Globe\\Lidar_data\\Onshore_testing\\LOS_calibration\\joined\\Mike\\globe_loscal_2000m_10min.csv',
             sep=';', decimal='.', header=True)